# L6: 检查输出结果

## 配置
#### 加载 API 密钥和相关 Python 库。
在本课程中，我们提供了一些为您加载 OpenAI API 密钥的代码

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = 'API_KEY'

In [2]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

### 检查输出结果是否包含潜在的有害内容。

In [3]:
final_response_to_customer = f"""
SmartX Pro手机具有6.1英寸的显示屏，128GB的存储空间，12MP的双摄像头和5G网络。\
FotoSnap DSLR相机具有24.2MP的传感器，1080p视频录制能力，3英寸LCD显示屏和可更换镜头。\
我们拥有各种电视产品，包括CineView 4K电视，配备55英寸的显示屏，4K分辨率，HDR和智能电视功能。\
此外，我们还有SoundMax家庭影院音响系统，拥有5.1声道，1000W输出功率，无线低音炮和蓝牙功能。\
您对这些产品或我们提供的其他产品有任何特定的问题吗？
"""
response = openai.Moderation.create(
    input=final_response_to_customer
)
moderation_output = response["results"][0]
print(moderation_output)

{
  "categories": {
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": false,
    "violence/graphic": false
  },
  "category_scores": {
    "hate": 1.3900634e-05,
    "hate/threatening": 2.4370496e-07,
    "self-harm": 4.5668235e-06,
    "sexual": 0.00011302941,
    "sexual/minors": 1.0577807e-06,
    "violence": 9.768882e-06,
    "violence/graphic": 2.34361e-06
  },
  "flagged": false
}


### 请检查输出是否根据提供的产品信息准确无误。

In [4]:
system_message = f"""
您是一个助理，负责评估客户服务代理的回答是否足够回答客户的问题，\
并验证助理从产品信息中引用的所有事实是否正确。
产品信息、用户和客户服务代理的消息将用3个反引号（```)分隔。
以单个字母作为回复，不带标点符号：
Y - 如果输出足够回答问题，并且回复正确使用了产品信息
N - 相反情况

只输出一个字母。
"""
customer_message = f"""
告诉我关于SmartX Pro手机和Fotosnap相机（其中包括DSLR型号）的信息。\
同时也告诉我关于你们的电视的情况。"""
product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

回答是否正确地使用了检索到的信息？
回答是否充分地回答了问题？

回答Y或者N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

Y


In [5]:
another_response = "生活就像一盒巧克力。"
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{another_response}```

回答是否正确地使用了检索到的信息？
回答是否充分地回答了问题？

回答Y或者N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages)
print(response)

N
